In [ ]:
# Datasets and Parquet

**TOC:**

1. Reading and writing Parquet
2. Resampling 15min data

## 1. Reading and writing Parquet

**Context: PRO and CON of Parquet** 

**PRO**<BR>
- saves data types, not like csv where everything is a string
- compression results in 2 upto 10 times smaller  than csv
- faster: Parquet store data in columns rather than rows, enabling selective reading only relevant data.

In [2]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import pyarrow

print(f'Python version: {sys.version}')
print(f'pandas version: {pd.__version__}')
print(f'numpy version: {np.__version__}')
print(f'pyarrow version: {pyarrow.__version__}')

Python version: 3.11.9 | packaged by conda-forge | (main, Apr 19 2024, 18:34:54) [Clang 16.0.6 ]
pandas version: 2.2.2
numpy version: 1.26.4
pyarrow version: 17.0.0


In [4]:
# Step 0: Define the paths
file_path_read = 'https://raw.githubusercontent.com/tribp/Forecasting/main/datasets/Elia/ods001.parquet'
file_path_save ='Elia_2015-2024.parquet'

# Step 1: Load the Parquet file into a DataFrame
df = pd.read_parquet(file_path_read)
df.head(2)

,datetime,resolutioncode,totalload,mostrecentforecast,mostrecentconfidence10,mostrecentconfidence90,dayaheadforecast,dayaheadconfidence10,dayaheadconfidence90,weekaheadforecast
0,2025-01-27 23:45:00+01:00,PT15M,NaN,9592.2,9180.97,10003.43,9592.2,9180.97,10003.43,9592.2
1,2025-01-27 23:30:00+01:00,PT15M,NaN,9642.1,9228.73,10055.47,9642.1,9228.73,10055.47,9642.1


In [ ]:
# Step 2: Ensure the 'Datetime' column is in datetime format
df['datetime'] = pd.to_datetime(df['datetime'])

# Step 3: Filter the DataFrame for rows until 1st January 2025
df_filtered = df[df['datetime'] < '2025-01-01']

# Step 4: Reset the index of the DataFrame
df_filtered.reset_index(drop=True, inplace=True)

# Step 5: Save the filtered DataFrame back to a new Parquet file
df_filtered.to_parquet(file_path_save)

In [15]:
df_filtered.head()

,datetime,resolutioncode,totalload,mostrecentforecast,mostrecentconfidence10,mostrecentconfidence90,dayaheadforecast,dayaheadconfidence10,dayaheadconfidence90,weekaheadforecast
0,2024-12-31 23:45:00+01:00,PT15M,8593.73,8258.90,8015.84,8501.96,8780.26,8325.54,9234.99,8645.52
1,2024-12-31 23:30:00+01:00,PT15M,8697.44,8364.12,8117.97,8610.27,8892.03,8431.52,9352.55,8759.77
2,2024-12-31 23:15:00+01:00,PT15M,8774.47,8445.72,8197.17,8694.27,8978.79,8513.78,9443.79,8852.59
3,2024-12-31 23:00:00+01:00,PT15M,8858.48,8542.78,8291.37,8794.19,9081.96,8611.61,9552.31,8952.74
4,2024-12-31 22:45:00+01:00,PT15M,8861.50,8982.02,8736.12,9227.92,9059.75,8599.72,9519.79,8925.19


In [16]:
df = pd.read_parquet(file_path_save)
df.head()

<bound method NDFrame.head of                         datetime resolutioncode  totalload  \
0      2024-12-31 23:45:00+01:00          PT15M    8593.73   
1      2024-12-31 23:30:00+01:00          PT15M    8697.44   
2      2024-12-31 23:15:00+01:00          PT15M    8774.47   
3      2024-12-31 23:00:00+01:00          PT15M    8858.48   
4      2024-12-31 22:45:00+01:00          PT15M    8861.50   
...                          ...            ...        ...   
350683 2015-01-01 01:00:00+01:00          PT15M    9755.00   
350684 2015-01-01 00:45:00+01:00          PT15M    9821.78   
350685 2015-01-01 00:30:00+01:00          PT15M    9952.87   
350686 2015-01-01 00:15:00+01:00          PT15M   10051.28   
350687 2015-01-01 00:00:00+01:00          PT15M   10142.19   

        mostrecentforecast  mostrecentconfidence10  mostrecentconfidence90  \
0                  8258.90                 8015.84                 8501.96   
1                  8364.12                 8117.97                 86

In [17]:
print(df.dtypes)
print(df_filtered.dtypes)

datetime                  datetime64[ms, Europe/Brussels]
resolutioncode                                     object
totalload                                         float64
mostrecentforecast                                float64
mostrecentconfidence10                            float64
mostrecentconfidence90                            float64
dayaheadforecast                                  float64
dayaheadconfidence10                              float64
dayaheadconfidence90                              float64
weekaheadforecast                                 float64
dtype: object
datetime                  datetime64[ms, Europe/Brussels]
resolutioncode                                     object
totalload                                         float64
mostrecentforecast                                float64
mostrecentconfidence10                            float64
mostrecentconfidence90                            float64
dayaheadforecast                                  float64


# 2. Resampling 15min data

## 2.1 We need a new file that contains all daily maxima of the 15min values

In [38]:
# Step 0: Define the paths
file_path_read = 'https://raw.githubusercontent.com/tribp/Forecasting/main/datasets/Elia/Elia_2015-2024.parquet'
file_path_save ='Elia_2015-2024_daily_max.parquet'

# Step 1: Load the Parquet file into a DataFramed
df_daily_max = pd.read_parquet(file_path_read)
#print(df_monthly.head(2))

# Step 2: Set the 'datetime' column as the index
df_daily_max.set_index('datetime', inplace=True)

# Step 3: delete all columns except 'datetime'(Index), 'totalload' and mostrecentforecast'
df_daily_max = df_daily_max[['totalload', 'mostrecentforecast']]

# Step 4: Resample the DataFrame to a daily frequency that holds the daily maximum
df_daily_max = df_daily_max.resample('D').max()
df_daily_max.head(2)

# Step 5: Save the resampled DataFrame back to a new Parquet file
df_daily_max.to_parquet(file_path_save)





## 2.2 Read and check daily maxima file

In [ ]:
# Step 0: Define the paths
file_path_read = 'https://raw.githubusercontent.com/tribp/Forecasting/main/datasets/Elia/Elia_2015-2024_daily_max.parquet'

# Step 1: Load the Parquet file into a DataFramed
df_daily_max = pd.read_parquet(file_path_read)
df_daily_max.head(2)